# GGUF Model Assistant in Google Colab


In diesem Notebook wird ein GGUF-Modell von Hugging Face geladen, eine Chat-Funktion erstellt, das Modell trainiert und die Ergebnisse in Google Drive und Hugging Face gespeichert.
Folge den Schritten, um das Projekt vollständig auszuführen.


In [ ]:

!pip install transformers huggingface_hub accelerate datasets llama-cpp-python


### Google Drive Mount (optional, falls Sie das Modell speichern möchten)

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


### Hugging Face API-Schlüssel eingeben

In [ ]:

from huggingface_hub import notebook_login
notebook_login()  # Du wirst nach deinem API-Schlüssel gefragt.


### GGUF-Modell laden

In [ ]:

from llama_cpp import Llama

# Beispiel: GGUF-Modell laden
model_path = "TheBloke/Mistral-7B-GGUF"  # Beispielmodell
model = Llama(model_path=model_path)


### Chat-Funktion erstellen

In [ ]:

def chat(prompt, model):
    response = model(prompt, max_tokens=100)
    return response['choices'][0]['text']

# Beispiel-Prompt
prompt = "User: Was sind die Vorteile von Mistral? Assistant:"
response = chat(prompt, model)
print(response)


### Trainingsdaten erstellen

In [ ]:

import json

data = [
    {"messages": [{"role": "system", "content": "Du bist ein hilfreicher Assistent"}, 
                  {"role": "user", "content": "Wie geht es dir?"}, 
                  {"role": "assistant", "content": "Mir geht es gut, danke."}]}
]

with open('train_data.jsonl', 'w') as f:
    for entry in data:
        f.write(json.dumps(entry) + '\n')


### Modell trainieren

In [ ]:

from transformers import Trainer, TrainingArguments

# Definiere die Trainingsargumente
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    logging_dir="./logs"
)

# Trainer initialisieren und trainieren
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data  # Ersetze durch dein Dataset
)

trainer.train()


### Modell speichern

In [ ]:

# Speichern in Google Drive
model.save_pretrained('/content/drive/MyDrive/gguf_model')

# Hochladen auf Hugging Face
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj='/content/drive/MyDrive/gguf_model',
    path_in_repo="gguf_model",
    repo_id="dein_username/modellname",
    repo_type="model"
)


### Testen des trainierten Modells

In [ ]:

test_prompt = "User: Was ist der Vorteil von Mistral?"
response = chat(test_prompt, model)
print(response)
